In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from mpl_toolkits.mplot3d import Axes3D

path = '../output_data/test_output/'

In [ ]:
with pd.HDFStore(path + 'X0_ffh.h5') as store:
    opd = store.select('dat_0')
opd.index = opd.index.droplevel(['eps', 'phi', 'ffh', 'test'])
opinions = opd.xs(level=1, key='opinions')
dfN = pd.DataFrame(index=opinions.index)
for i in range(8):
    dfN[f'N{i+1}'] = (opinions==i).sum(axis=1)
dfN_vis = pd.DataFrame(index=dfN.index)
dfN_vis['x'] = dfN['N1'] + dfN['N2'] + dfN['N3']
dfN_vis['y'] = dfN['N4'] + dfN['N5'] + dfN['N6']
dfN_vis['z'] = dfN['N7'] + dfN['N8']

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=dfN_vis[['x']], ys=dfN_vis[['y']], zs=dfN_vis['z'])

In [ ]:
with pd.HDFStore(path + 'X0_ffh.h5') as store:
    trj = store.select('dat_1')
trj.index = trj.index.droplevel(['eps', 'phi', 'ffh', 'test'])
trj.head()

In [ ]:
cond = pd.DataFrame(index=trj.index)
cond['a'] = trj['K_c']/trj['K_d'] - 0.15276374882897656
cond['b'] = trj['i_c']
cond['c'] = trj['r_c']
cond.head()

In [ ]:
t_max=9

# calculate point distance
cond['d'] = np.sqrt((cond['a'])**2)

# sum over point distance for each trajectory
dfN['distance'] = cond['d'].unstack('sample').loc[:t_max].sum(axis=0)

# map distances to interval (0,1]
dfN['inv_distance'] = 1./(1. + dfN['distance'])

# use 1/distance as weight
dfN['weight'] = 1./cond['d'].unstack('sample').loc[:t_max].sum(axis=0)
dfN['weight'] = dfN['weight']/sum(dfN['weight'].values)

for s in dfN.index.values:
    cond.loc[s, 'inv_distance'] = dfN.loc[s, 'inv_distance']
dfN['weight'].sum()
dfN.head()

In [ ]:
from collections import Counter

fig, axes = plt.subplots(ncols=3, figsize=(8, 2.8))

def plot_cond(df, c, ax, t_max, thresh=0.5):
    tmp = df
    tmp1 = tmp[[c]].loc[tmp['inv_distance']<thresh]
    tmp2 = tmp[[c]].loc[tmp['inv_distance']>thresh]
    try:
        tmp1.unstack('sample').loc[:t_max].plot(ax=ax, alpha=.03, legend=False, color=['k'])
    except TypeError:
        print('no data')
    try:
        tmp2.unstack('sample').loc[:t_max].plot(ax=ax, legend=False, alpha=.7, color=['orange'])
    except TypeError:
        print('no data')
    
thresh=0.5

plot_cond(cond, 'a', axes[0], 20, thresh=thresh)
plot_cond(cond, 'b', axes[1], 20, thresh=thresh)

try:
    dfN['inv_distance'].loc[dfN['inv_distance']>thresh].plot.hist(ax=axes[2], color=['orange'], alpha=0.9, bins=0.1*np.arange(5,11))
    dfN['inv_distance'].loc[dfN['inv_distance']<thresh].plot.hist(ax=axes[2], color=['k'], alpha=0.2, bins=0.1*np.arange(0,6))
except TypeError:
    print('no data')

#plt.yscale('log', nonposy='clip')

# dfN['weight'].loc[dfN['weight']<thresh].plot(ax=axes[2], style='.', alpha=0.3, color='k')
# dfN['weight'].loc[dfN['weight']>thresh].plot(ax=axes[2], style='.', color='orange')

for ax in axes[:2]:
    ax.set_xlim([0,20])
    ax.set_xlabel('time')
    ax.axvline(t_max, color='k')

axes[0].set_ylabel(r'$K_c/K_d - K_{c0}/K_{d0}$')
axes[1].set_ylabel('fraction of clean investment')
axes[1].set_ylim((0,1.03))
axes[2].set_ylabel('count')
axes[2].set_xlabel(r'$1/(1+d)$')
axes[2].set_xlim([0,1])

fig.tight_layout()
fig.savefig('initial_condition_sampling.pdf')

In [ ]:
oNames = [f'N{i+1}' for i in range(8)]

dfN_norm = pd.DataFrame(index=dfN.index, columns=dfN.columns)

for i in range(len(dfN)):
    dfN_norm.loc[i, oNames] = dfN.loc[i, oNames] * dfN['weight'][i]
dfN_norm['weight'] = dfN['weight']
print([round(x) for x in dfN_norm.sum()])
print(sum([round(x) for x in dfN_norm[oNames].sum()]))

In [ ]:
sum([15, 8, 12, 12, 14, 17, 7, 16])

In [ ]:
# initial opinions:
fitted_opinions_distribution = [15, 8, 12, 12, 14, 17, 7, 16]
x = [2 * x for x in fitted_opinions_distribution]
opinions = []

for i, xi in enumerate(x):
    opinions += int(np.round(xi)) * [i]
np.random.shuffle(opinions)
# as the fitted distribution has one count to many, I have to leave two out
# here.
opinions = opinions[:-2]

In [ ]:
from collections import Counter
cnt = Counter(opinions)
for i in range(7):
    print(i, cnt[i])
print(cnt.keys())
print(cnt.values())

In [ ]:
0 30
1 16
2 24
3 24
4 27
5 34
6 14
dict_keys([5, 2, 7, 0, 6, 3, 4, 1])
dict_values([34, 24, 31, 30, 14, 24, 27, 16])


In [ ]:
with pd.HDFStore(path + 'X0a_ffh.h5') as store:
    opd = store.select('dat_0')
opd.index = opd.index.droplevel(['eps', 'phi', 'ffh', 'test'])
opinions = opd.xs(level=1, key='opinions')
dfN = opinions
dfN_vis = pd.DataFrame(index=dfN.index)
dfN_vis['x'] = dfN['O1'] + dfN['O2'] + dfN['O3']
dfN_vis['y'] = dfN['O4'] + dfN['O5'] + dfN['O6']
dfN_vis['z'] = dfN['O7'] + dfN['O8']

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=dfN_vis[['x']], ys=dfN_vis[['y']], zs=dfN_vis['z'])
dfN.head()